In [1]:
import json
import os
from dotenv import load_dotenv
from tqdm import tqdm

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma 

# --- 1. 환경 설정 ---
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

# --- 2. 파일 및 DB 경로 설정 ---
INPUT_JSONL = "./output/rag_data.jsonl"
DB_PERSIST_DIRECTORY = "./db/chroma_db" # 이 폴더는 삭제된 상태여야 함

# --- 3. [수정] 수동으로 메타데이터 정제하는 함수 ---
def clean_metadata_for_chroma(metadata):
    """
    ChromaDB가 list나 None을 저장할 수 있도록 수동으로 변환합니다.
    """
    clean_meta = {}
    if not isinstance(metadata, dict):
        return {}
        
    for key, value in metadata.items():
        if isinstance(value, list):
            # 1. list -> 콤마(,)로 구분된 단일 문자열로 변환
            # (예: ["A", "B"] -> "A, B")
            clean_meta[key] = ", ".join(map(str, value))
        elif value is None:
            # 2. None -> 빈 문자열("")로 변환
            clean_meta[key] = ""
        elif isinstance(value, (str, int, float, bool)):
            # 3. 허용되는 타입은 그대로 둠
            clean_meta[key] = value
        else:
            # 4. 그 외 복잡한 타입(dict 등)은 문자열로 강제 변환
            clean_meta[key] = str(value)
    return clean_meta

# --- 4. 문서 로드 및 '수동 정제' ---
print(f"'{INPUT_JSONL}' 파일에서 문서를 로드합니다...")

all_documents = [] # Document 객체 리스트
all_ids = []       # 고유 ID 리스트

try:
    with open(INPUT_JSONL, "r", encoding="utf-8") as f:
        for line in tqdm(f, desc="Loading and splitting data"):
            try:
                data = json.loads(line)
                
                page_content = data.pop('rag_text', None) 
                if not page_content: continue
                    
                doc_id = str(data.get('tmdb_id'))
                if not doc_id: continue
                
                # [수정] 'filter_complex_metadata' 대신 수동 정제 함수 사용
                metadata = clean_metadata_for_chroma(data)
                
                doc = Document(page_content=page_content, metadata=metadata)
                
                all_documents.append(doc)
                all_ids.append(doc_id)

            except json.JSONDecodeError:
                print(f"Skipping malformed JSON line: {line[:50]}...")
                
except FileNotFoundError:
    print(f"오류: '{INPUT_JSONL}' 파일을 찾을 수 없습니다.")
    exit()
    
print(f"총 {len(all_documents)}개의 '정제된' 문서를 RAG 색인을 위해 준비했습니다.")


# --- 5. 임베딩 및 Vector Store '배치' 생성 (이전과 동일) ---
if all_documents:
    print("임베딩 모델을 초기화합니다 (OpenAI: text-embedding-3-small)...")
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    print(f"ChromaDB를 '{DB_PERSIST_DIRECTORY}' 경로에 '새로' 생성합니다...")
    
    db = Chroma(
        persist_directory=DB_PERSIST_DIRECTORY,
        embedding_function=embeddings
    )

    batch_size = 100 
    print(f"총 {len(all_documents)}개의 '정제된' 문서를 {batch_size}개씩 배치로 나누어 추가합니다.")

    for i in tqdm(range(0, len(all_documents), batch_size), desc="Indexing batches"):
        
        batch_docs = all_documents[i:i + batch_size]
        batch_ids = all_ids[i:i + batch_size]
        
        # 이제 '정제된' 문서가 전달되므로 list/None 오류가 발생하지 않습니다.
        db.add_documents(
            documents=batch_docs,
            ids=batch_ids
        )
    
    # db.persist()는 최신 버전에선 필요 없으므로 제거
    print("모든 배치를 완료했습니다. DB가 디스크에 자동 저장되었습니다.")
    
    print(f"\n✅ 완료! {len(all_documents)}개의 문서가 임베딩되어 ChromaDB에 저장되었습니다.")
    print(f"DB 경로: {DB_PERSIST_DIRECTORY}")
    print(f"현재 컬렉션의 총 항목 수: {db._collection.count()}")
    
else:
    print("임베딩할 문서가 없습니다.")

'./output/rag_data.jsonl' 파일에서 문서를 로드합니다...


Loading and splitting data: 860it [00:00, 39106.88it/s]

총 860개의 '정제된' 문서를 RAG 색인을 위해 준비했습니다.
임베딩 모델을 초기화합니다 (OpenAI: text-embedding-3-small)...


ChromaDB를 './db/chroma_db' 경로에 '새로' 생성합니다...
총 860개의 '정제된' 문서를 100개씩 배치로 나누어 추가합니다.


Indexing batches: 100%|██████████| 9/9 [00:24<00:00,  2.71s/it]


모든 배치를 완료했습니다. DB가 디스크에 자동 저장되었습니다.

✅ 완료! 860개의 문서가 임베딩되어 ChromaDB에 저장되었습니다.
DB 경로: ./db/chroma_db
현재 컬렉션의 총 항목 수: 860


In [10]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import pprint # 메타데이터 예쁘게 출력

# --- 1. 환경 설정 ---
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

# --- 2. DB 로드 ---
DB_PERSIST_DIRECTORY = "./db/chroma_db"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print(f"'{DB_PERSIST_DIRECTORY}'에서 DB를 로드합니다...")

try:
    db = Chroma(
        persist_directory=DB_PERSIST_DIRECTORY,
        embedding_function=embeddings
    )
    print(f"✅ DB 로드 완료. 총 {db._collection.count()}개의 항목이 있습니다.")
except Exception as e:
    print(f"❌ DB 로드 실패: {e}")
    print("DB 폴더가 손상되었거나 경로가 다를 수 있습니다.")
    exit()


# --- 3. 유사도 검색 테스트 ---
# '승부' (이병헌, 유아인 주연의 바둑 영화)의 줄거리를 기반으로 쿼리
test_query = "스승과 제자가 바둑으로 대결하는 영화"

print(f"\n--- 쿼리 테스트 ---")
print(f"쿼리: '{test_query}'")

try:
    # .similarity_search_with_score: 유사도 점수와 함께 Document 객체를 반환
    # k=3 : 상위 3개 결과
    docs_with_scores = db.similarity_search_with_score(test_query, k=3)

    if docs_with_scores:
        print(f"\n[검색 결과 (상위 {len(docs_with_scores)}개)]")
        
        for i, (doc, score) in enumerate(docs_with_scores):
            print(f"\n--- 결과 {i+1} (유사도 점수: {score:.4f}) ---")
            
            metadata = doc.metadata
            
            # --- 4. 메타데이터 확인 ---
            print(f"  제목: {metadata.get('title_ko', '!!! 제목 없음 !!!')}")
            print(f"  연도: {metadata.get('year', '!!! 연도 없음 !!!')}")
            
            # [!!] filter_complex_metadata로 인해 이 필드들은 'None'일 것입니다.
            print(f"  장르: {metadata.get('genres', '!!! 장르 정보 없음 !!!')}")
            print(f"  출연: {metadata.get('cast', '!!! 출연 정보 없음 !!!')}")
            print(f"  OTT: {metadata.get('ott_streaming_kr', '!!! OTT 정보 없음 !!!')}")

            print("\n  [전체 메타데이터 확인용]")
            pprint.pprint(metadata)
            
    else:
        print("검색 결과가 없습니다.")
        
except Exception as e:
    print(f"❌ 검색 중 오류 발생: {e}")

'./db/chroma_db'에서 DB를 로드합니다...
✅ DB 로드 완료. 총 860개의 항목이 있습니다.

--- 쿼리 테스트 ---
쿼리: '스승과 제자가 바둑으로 대결하는 영화'

[검색 결과 (상위 3개)]

--- 결과 1 (유사도 점수: 1.1526) ---
  제목: 잡아야 산다
  연도: 2016
  장르: 액션, 코미디
  출연: 김승우, 김정태, 한상혁, Shin Kang-Woo, 김민규, Moon Yong-seok, Choi Ho-joong, Seo Beom-sik, 오만석
  OTT: Watcha, TVING

  [전체 메타데이터 확인용]
{'cast': '김승우, 김정태, 한상혁, Shin Kang-Woo, 김민규, Moon Yong-seok, Choi Ho-joong, '
         'Seo Beom-sik, 오만석',
 'directors': '오인천',
 'genres': '액션, 코미디',
 'imdb_id': 'tt5374830',
 'keywords': '',
 'ott_streaming_kr': 'Watcha, TVING',
 'overview': '잘나가는 CEO 쌍칼 승주와 강력계 허탕 형사 정택. 20년 지기 친구는 개뿔! 서로 으르렁 거리기만 하던 두 '
             '사람이 웬일로 의기투합했다. 개념 따위는 시원하게 말아드신 고딩 4인방에게 퍽치기 당해 지갑과 핸드폰까지 몽땅 털린 '
             '승주. 건수 하나 잡을까 얼떨결에 끼어들었다가 띠동갑도 넘는 고딩들에게 총까지 뺏긴 정택. 목숨같은 물건까지 털리고 '
             '개망신 제대로 당한 형님들과 달밤에 형님들 똥개 훈련시키는 고딩 4인방의 예측 불허 추격전이 펼쳐지는데...',
 'qid': 'http://www.wikidata.org/entity/Q21710400',
 'rating_kr': '',
 'title_en': 'Catch Him to Survive',
 'title_ko': '잡아야 산다',
 '

In [16]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import pprint

# --- 1. 환경 설정 ---
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

# --- 2. DB 로드 ---
DB_PERSIST_DIRECTORY = "./db/chroma_db"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print(f"'{DB_PERSIST_DIRECTORY}'에서 DB를 로드합니다...")

try:
    db = Chroma(
        persist_directory=DB_PERSIST_DIRECTORY,
        embedding_function=embeddings
    )
    # 네이티브 컬렉션을 변수에 할당
    collection = db._collection
    print(f"✅ DB 로드 완료. 총 {collection.count()}개의 항목이 있습니다.")
except Exception as e:
    print(f"❌ DB 로드 실패: {e}")
    exit()

# --- 3. 쿼리 테스트 ---
pp = pprint.PrettyPrinter(indent=2)

def run_query(query_text, k=3, where_filter=None, where_document_filter=None):
    """
    네이티브 collection.query()를 사용해 모든 유형의 쿼리를 실행합니다.
    """
    print("="*50)
    print(f"시맨틱 쿼리: '{query_text}'")
    if where_filter:
        print(f"메타데이터 필터(where): {where_filter}")
    if where_document_filter:
        print(f"본문 필터(where_document): {where_document_filter}")
    print("="*50)

    try:
        # LangChain의 .similarity_search() 대신 네이티브 .query() 사용
        
        # 1. 쿼리 텍스트를 벡터로 변환
        query_embeddings = embeddings.embed_query(query_text)
        
        # 2. DB에 쿼리
        results = collection.query(
            query_embeddings=[query_embeddings], # 시맨틱 검색 기준
            n_results=k,
            where=where_filter,                 # 메타데이터 필터
            where_document=where_document_filter  # [핵심] 본문 필터
        )

        if results and results.get('documents'):
            # 결과가 복잡한 리스트의 리스트로 반환됨
            results_list = zip(
                results['ids'][0],
                results['documents'][0],
                results['metadatas'][0],
                results['distances'][0]
            )
            
            for i, (doc_id, doc_content, metadata, distance) in enumerate(results_list):
                print(f"\n--- 결과 {i+1} (유사도 거리: {distance:.4f}) ---")
                print(f"  ID: {doc_id}")
                print(f"  제목: {metadata.get('title_ko')}")
                print(f"  장르: {metadata.get('genres')}")
                print(f"  출연: {metadata.get('cast')}")
                
                # (메타데이터 전체가 궁금할 경우 주석 해제)
                # print("\n  [전체 메타데이터 확인용]")
                # pp.pprint(metadata)
        else:
            print("검색 결과가 없습니다.")
            
    except Exception as e:
        print(f"❌ 검색 중 오류 발생: {e}")

# --- 
# [테스트 1: "이병헌"이 나오는 영화 (본문 필터)]
# 쿼리: "영화" (일반적)
# 필터: rag_text에 "이병헌"이 포함된 것
# ---
run_query(
    query_text="영화",
    where_document_filter={"$contains": "이병헌"} # rag_text의 [주요 출연진] 검색
)


# --- 
# [테스트 2: "승부" (시맨틱 검색)]
# "승부"가 1위로 나와야 합니다.
# ---
run_query(
    query_text="이병헌, 유아인 바둑 영화"
)


# --- 
# [테스트 3: "박정민"이 나오는 "2025년" 영화 (본문 + 메타데이터 필터)]
# 쿼리: "영화" (일반적)
# 필터 1: rag_text에 "박정민" 포함
# 필터 2: metadata의 year가 2025
# ---
run_query(
    query_text="영화",
    where_filter={"year": {"$eq": 2025}},           # 메타데이터 필터
    where_document_filter={"$contains": "박정민"}   # 본문 필터
)

'./db/chroma_db'에서 DB를 로드합니다...
✅ DB 로드 완료. 총 860개의 항목이 있습니다.
시맨틱 쿼리: '영화'
본문 필터(where_document): {'$contains': '이병헌'}

--- 결과 1 (유사도 거리: 1.2708) ---
  ID: 348689
  제목: 협녀, 칼의 기억
  장르: 드라마, 역사, 액션
  출연: 이병헌, 전도연, 김고은, 이준호, 이경영, 김태우, 김수안, 김영민, 성유빈, Guzal Tursunova

--- 결과 2 (유사도 거리: 1.2750) ---
  ID: 626872
  제목: 비상선언
  장르: 액션, 드라마, 스릴러
  출연: 송강호, 이병헌, 전도연, 김남길, 임시완, 김소진, 박해준, 도영서, 현봉식, 우미화

--- 결과 3 (유사도 거리: 1.3073) ---
  ID: 567646
  제목: 극한직업
  장르: 액션, 코미디, 범죄
  출연: 류승룡, 이하늬, 진선규, 이동휘, 공명, 신하균, 오정세, 김의성, 송영규, 허준석
시맨틱 쿼리: '이병헌, 유아인 바둑 영화'

--- 결과 1 (유사도 거리: 1.0531) ---
  ID: 335157
  제목: 비치하트애솔
  장르: 로맨스
  출연: Kwon Hyun-sang, Lee Ga-heun, 한사명, 박명신, Kim Dong-wan, Yoon Yeo-jin, 이민지, Jeon Ji-hee, 서준영, 이나라

--- 결과 2 (유사도 거리: 1.0981) ---
  ID: 606740
  제목: 소리도 없이
  장르: 드라마, 범죄
  출연: 유아인, 유재명, 문승아, 조하석, 김자영, Kim Han-na, 서동수, 이가은혜, 유성주, Im Kang-sung

--- 결과 3 (유사도 거리: 1.1293) ---
  ID: 611667
  제목: 낙인
  장르: SF, 스릴러, 미스터리
  출연: Yang Ji, Yoon Ha-bin, Han Sung-min, Jang Tae-young, Eun Hyun-gil, 